# QRNG Demo Notebook 

### Full **QRNG** workflow:

**A. Setup & Overview**  
**B. Generate Quantum Bits** (Hⁿ + measure)  
**C. Visualize Counts & Bit-Frequency**  
**D. Test Fairness (Chi-square, KL)**  
**E. Mitigate Readout Errors (toggleable)**  
**F. Certification (LGI + NSIT, single-qubit)** *(optional)*  
**G. Password Mode (Base94 / custom alphabets)**  
**H. Reproducibility & Metadata**

Data/plots are saved into `/data/`.  
Modules are imported from `/src/`.

## A) Setup & Overview

- Ensures `../data/` exists for outputs
- Adds `../src/` to the Python path
- Imports the project modules:
  - `qrng` (core engine: H^n + bit cache + unbiased `uniform_int`)
  - `metrics` (histograms, X², KL, bit-frequency)
  - `mitigation` (readout calibration and correction)
  - `lgi_cert` (LGI + NSIT certified RNG demo)
  - `viz` (matplotlib helpers)

Point `BitPool` to a real backend (Qiskit Runtime) if available. By default, this uses `qasm_simulator`.

In [1]:
# A) Setup & Overview
import sys
from pathlib import Path

# Resolve project paths assuming this notebook sits in `notebooks/`
NB_DIR = Path.cwd()
ROOT_DIR = (NB_DIR / "..").resolve()
SRC_DIR = ROOT_DIR / "src"
DATA_DIR = ROOT_DIR / "data"

# Make sure imports work
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# Create ../data if missing
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Imports
from src import qrng, metrics, mitigation, lgi_cert, viz

import numpy as np
import json
import math
import matplotlib.pyplot as plt

print("Notebook directory:", NB_DIR)
print("Project root:", ROOT_DIR)
print("Data directory:", DATA_DIR)
print("Src directory:", SRC_DIR)

Notebook directory: /Users/wei/Desktop/QCode/QRNG
Project root: /Users/wei/Desktop/QCode
Data directory: /Users/wei/Desktop/QCode/data
Src directory: /Users/wei/Desktop/QCode/src


## B) Generate Quantum Bits

We build an *n*-qubit circuit with Hadamards (Hⁿ), measure, and gather raw bitstrings.

- Use `qrng.generate_bitstrings(n_qubits, shots)` for raw outcomes.
- Or use the cached `qrng.BitPool(...)` for efficient bit/uint / uniform integers.

Below, we:
1. Generate a batch of bitstrings for `n_qubits=4`.
2. Convert them to a histogram (counts over 0..15).
3. Save the counts JSON for later steps.

In [2]:
# B) Generate Quantum Bits
n_qubits = 4
shots = 10000
seed = 42  # for deterministic shuffling on simulators

bitstrings = qrng.generate_bitstrings(n_qubits=n_qubits, shots=shots, seed_simulator=seed)
len(bitstrings), bitstrings[:8]

(10000, ['0100', '0100', '1000', '0100', '0010', '1101', '0011', '0110'])

In [3]:
# Convert bitstrings -> histogram over integers (0..2^n - 1)
counts = {}
for s in bitstrings:
    k = int(s, 2)
    counts[k] = counts.get(k, 0) + 1

# Save counts to data/
counts_path = DATA_DIR / f"counts_{n_qubits}q_{shots}.json"
with open(counts_path, "w") as f:
    json.dump({str(k): int(v) for k, v in counts.items()}, f, indent=2)

print("Saved counts to:", counts_path)
sorted(list(counts.items()))[:8]

Saved counts to: /Users/wei/Desktop/QCode/data/counts_4q_10000.json


[(0, 589),
 (1, 662),
 (2, 633),
 (3, 625),
 (4, 632),
 (5, 618),
 (6, 692),
 (7, 638)]

## C) Visualize Counts & Bit-Frequency

Use `viz.plot_counts_histogram` and `metrics.bit_frequency` to get a quick sense of uniformity.

- **Counts histogram** should look roughly flat for a good RNG.
- **Bit-frequency** should be ~50% zeros and ~50% ones.

In [4]:
# C) Visualize
# 1) Counts histogram
fig, ax = viz.plot_counts_histogram(counts, title=f"{n_qubits}-qubit counts ({shots} shots)")
fig_path = DATA_DIR / f"hist_{n_qubits}q_{shots}.png"
fig.savefig(fig_path, dpi=160, bbox_inches="tight")
plt.close(fig)
print("Saved histogram to:", fig_path)

# 2) Bit-frequency
bf = metrics.bit_frequency(bitstrings)
fig, ax = viz.plot_bit_frequency(bf)
bf_path = DATA_DIR / f"bitfreq_{n_qubits}q_{shots}.png"
fig.savefig(bf_path, dpi=160, bbox_inches="tight")
plt.close(fig)
print("Saved bit-frequency plot to:", bf_path)
bf

Saved histogram to: /Users/wei/Desktop/QCode/data/hist_4q_10000.png
Saved bit-frequency plot to: /Users/wei/Desktop/QCode/data/bitfreq_4q_10000.png


BitFreq(total_bits=40000, ones=20081, zeros=19919, frac_one=0.502025, frac_zero=0.497975)

## D) Test Fairness (Chi-square, KL)

- **Chi-square vs. uniform**: compares observed counts with ideal uniform counts.
- **KL divergence**: compares observed distribution to uniform (in bits).

> Lower is better for both metrics. For χ², you may also look at the `p-value` (requires SciPy).

In [5]:
# D) Test Fairness
support_size = 2 ** n_qubits

# Chi-square
chi2 = metrics.chi_square_uniform(counts, support_size=support_size)

# KL divergence: compare observed (normalized) vs uniform
obs_vec = metrics.counts_to_vector({int(k): int(v) for k, v in counts.items()}, support_size)
p_obs = obs_vec / obs_vec.sum()
p_uni = np.ones_like(p_obs) / p_obs.size
kl = metrics.kl_divergence(p_obs, p_uni)

# Save metrics
chi_path = DATA_DIR / f"chi2_{n_qubits}q_{shots}.json"
with open(chi_path, "w") as f:
    json.dump({"stat": chi2.stat, "df": chi2.df, "pvalue": chi2.pvalue}, f, indent=2)

kl_path = DATA_DIR / f"kl_{n_qubits}q_{shots}.json"
with open(kl_path, "w") as f:
    json.dump({"kl_bits": kl}, f, indent=2)

print("χ²:", chi2.stat, "df:", chi2.df, "p:", chi2.pvalue)
print("KL divergence (bits):", kl)

χ²: 20.64 df: 15 p: 0.14873108610488828
KL divergence (bits): 0.0014826087665039192


In [6]:
# Residuals plot (observed - expected)
expected = np.ones(support_size) * (shots / support_size)
fig, ax = viz.plot_uniformity_residuals(observed=obs_vec, expected=expected,
                                        title=f"Uniformity residuals ({n_qubits}q, {shots} shots)")
resid_path = DATA_DIR / f"residuals_{n_qubits}q_{shots}.png"
fig.savefig(resid_path, dpi=160, bbox_inches="tight")
plt.close(fig)
print("Saved residual plot to:", resid_path)

Saved residual plot to: /Users/wei/Desktop/QCode/data/residuals_4q_10000.png


## E) Mitigate Readout Errors (toggleable)

This step demonstrates **LITE** readout error mitigation:
1. Calibrate **per-qubit** assignment matrices \(A_q\) (2×2).
2. Build the global assignment matrix \(A = \bigotimes_q A_q\).
3. Correct the observed distribution \(p_{\text{ideal}} \approx A^{-1} p_{\text{noisy}}\).

> For teaching-scale demos (2–5 qubits), full-kron is fine. For larger *n*, consider local schemes.

In [7]:
# E) Mitigation
# 1) Calibrate per-qubit assignment matrices on the current backend
backend = qrng._QASM_BACKEND  # or pass a real backend
per_qubit = [mitigation.calibrate_qubit_assignment(q, backend, shots=8000) for q in range(n_qubits)]

# 2) Build global assignment matrix
A = mitigation.build_global_assignment(per_qubit)

# 3) Correct the observed probability vector
p_corr = mitigation.apply_correction(p_obs, A)

# Save corrected vector
npz_path = DATA_DIR / f"mitigated_{n_qubits}q_{shots}.npz"
np.savez(npz_path, p_noisy=p_obs, p_corrected=p_corr, A=A)
print("Saved corrected probabilities & A to:", npz_path)

# Re-compute fairness after mitigation
kl_corr = metrics.kl_divergence(p_corr, p_uni)
chi2_corr_stat = float(np.sum((p_corr * shots - expected) ** 2 / expected))

# Save post-mitigation metrics
with open(DATA_DIR / f"chi2_after_{n_qubits}q_{shots}.json", "w") as f:
    json.dump({"stat": chi2_corr_stat, "df": support_size - 1, "pvalue": float("nan")}, f, indent=2)
with open(DATA_DIR / f"kl_after_{n_qubits}q_{shots}.json", "w") as f:
    json.dump({"kl_bits": kl_corr}, f, indent=2)

print("After mitigation:")
print("  KL (bits):", kl_corr)
print("  χ² (stat only):", chi2_corr_stat)

Saved corrected probabilities & A to: /Users/wei/Desktop/QCode/data/mitigated_4q_10000.npz
After mitigation:
  KL (bits): 0.0014826087665039192
  χ² (stat only): 20.640000000000004


## F) Certification (LGI + NSIT)

*(Optional)* Show **temporal** certification signals from a single-qubit protocol:

- **LGI \(K_3 = C_{12} + C_{23} - C_{13}\)**  
  Macrorealist bound \(K_3 \le 1\). Quantum can reach ≈ 1.5 for certain angles.
- **NSIT @ t2**: check if measuring at t1 changes the t2 marginal.

These are *indicators* and educational; not full device-independent proofs.

In [8]:
# F) Certification
theta = 0.4  # try 0.3–0.6 for visible effects
shots_cert = 8000

lgi = lgi_cert.run_lgi_k3(backend=qrng._QASM_BACKEND, shots=shots_cert, theta=theta, seed_simulator=seed)
nsit = lgi_cert.run_nsit_t2(backend=qrng._QASM_BACKEND, shots=shots_cert, theta=theta, seed_simulator=seed)

# Save JSON
with open(DATA_DIR / f"lgi_theta{theta}_{shots_cert}.json", "w") as f:
    json.dump({"K3": lgi.K3, "C12": lgi.C12, "C23": lgi.C23, "C13": lgi.C13, "violated": lgi.violated}, f, indent=2)
with open(DATA_DIR / f"nsit_theta{theta}_{shots_cert}.json", "w") as f:
    json.dump({"delta": nsit.delta, "p_with": nsit.p_with, "p_without": nsit.p_without}, f, indent=2)

# Plots
fig, ax = viz.plot_lgi_components(C12=lgi.C12, C23=lgi.C23, C13=lgi.C13, K3=lgi.K3,
                                  title=f"LGI components (θ={theta}, shots={shots_cert})")
fig.savefig(DATA_DIR / f"lgi_components_theta{theta}_{shots_cert}.png", dpi=160, bbox_inches="tight")
plt.close(fig)

fig, ax = viz.plot_nsit_delta(delta=nsit.delta, p_with=nsit.p_with, p_without=nsit.p_without,
                              title=f"NSIT @ t2 (θ={theta}, shots={shots_cert})")
fig.savefig(DATA_DIR / f"nsit_theta{theta}_{shots_cert}.png", dpi=160, bbox_inches="tight")
plt.close(fig)

lgi, nsit

(LGIResult(K3=1.4355, C12=0.709, C23=0.705, C13=-0.0215, violated=True),
 NSITResult(delta=0.528, p_with=(0.751625, 0.248375), p_without=(0.487625, 0.512375)))

## G) Password Mode (Base94 / Custom Alphabets)

Turn unbiased integers into human-usable passwords.

- Default alphabet: **Base94** (printable ASCII).
- Custom alphabets allowed (e.g., alnum only).
- Entropy estimate: \(H = \text{length} \times \log_2(|\text{alphabet}|)\).

In [9]:
# G) Password Mode
from src.quantum_passwords import PasswordGenerator, BASE94, estimate_entropy_bits

# Default Base94 generator
gen = PasswordGenerator()  # uses default pool
pwd_len = 16
passwords = gen.passwords(length=pwd_len, count=5)

# Entropy curve for README: 8..24 chars
lengths = list(range(8, 25, 2))
fig, ax = viz.plot_password_entropy_curve(lengths=lengths, alphabet_size=len(BASE94),
                                          title="Password entropy vs length (Base94)")
entropy_curve_path = DATA_DIR / "password_entropy_base94.png"
fig.savefig(entropy_curve_path, dpi=160, bbox_inches="tight")
plt.close(fig)

# Save sample passwords (DO NOT use in production)
pw_path = DATA_DIR / f"sample_passwords_base94_len{pwd_len}.txt"
with open(pw_path, "w") as f:
    for p in passwords:
        f.write(p + "\n")

print("Sample passwords:", passwords)
print("Entropy estimate (~bits):", round(estimate_entropy_bits(pwd_len), 2))
print("Saved entropy curve to:", entropy_curve_path)
print("Saved sample passwords to:", pw_path)

Sample passwords: ['CFQc5;A"Eaq-t,fG', '-0IX"|CyK6P1POza', 'aV!fe%\\[f4r`7\\XG', '\\n6ed#K^R1~.T^~;', '2)`)nL`kb,d{@~C6']
Entropy estimate (~bits): 104.87
Saved entropy curve to: /Users/wei/Desktop/QCode/data/password_entropy_base94.png
Saved sample passwords to: /Users/wei/Desktop/QCode/data/sample_passwords_base94_len16.txt


## H) Reproducibility & Metadata

We save a small JSON bundle with:
- Qiskit & numpy versions
- Backend name
- Parameters used in this run
- Notebook path (for context)

In [10]:
# H) Reproducibility & Metadata
import qiskit
meta = {
    "notebook": str(NB_DIR),
    "root_dir": str(ROOT_DIR),
    "data_dir": str(DATA_DIR),
    "src_dir": str(SRC_DIR),
    "params": {
        "n_qubits": n_qubits,
        "shots": shots,
        "seed": seed,
        "theta_cert": theta,
        "shots_cert": shots_cert,
        "password_len_demo": pwd_len,
    },
    "versions": {
        "python": sys.version,
        "numpy": np.__version__,
        "qiskit": qiskit.__version__ if hasattr(qiskit, "__version__") else "unknown",
        "matplotlib": plt.matplotlib.__version__ if hasattr(plt, "matplotlib") else "unknown",
    },
    "backend": getattr(qrng._QASM_BACKEND, "name", str(qrng._QASM_BACKEND)),
}

meta_path = DATA_DIR / "run_meta.json"
with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2)

print("Saved run metadata to:", meta_path)
meta

Saved run metadata to: /Users/wei/Desktop/QCode/data/run_meta.json


{'notebook': '/Users/wei/Desktop/QCode/QRNG',
 'root_dir': '/Users/wei/Desktop/QCode',
 'data_dir': '/Users/wei/Desktop/QCode/data',
 'src_dir': '/Users/wei/Desktop/QCode/src',
 'params': {'n_qubits': 4,
  'shots': 10000,
  'seed': 42,
  'theta_cert': 0.4,
  'shots_cert': 8000,
  'password_len_demo': 16},
 'versions': {'python': '3.13.7 (v3.13.7:bcee1c32211, Aug 14 2025, 19:10:51) [Clang 16.0.0 (clang-1600.0.26.6)]',
  'numpy': '2.3.4',
  'qiskit': '2.2.1',
  'matplotlib': '3.10.7'},
 'backend': 'qasm_simulator'}